# Face Recognition Project

In [207]:
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import dlib

## Deep Learning CNN model to recognize face

### This script uses a database of images and creates CNN model on top of it to test if the given image is recognized correctly or not
 
 
 
### IMAGE PRE-PROCESSING for TRAINING and TESTING data:

In [208]:
# Specifying the folder where images are present
TrainingImagePath='/Users//sruja/PycharmProjects/pythonProject/test1'
 
from keras.preprocessing.image import ImageDataGenerator
# Defining pre-processing transformations on raw images of training data
# These hyper parameters help to generate slightly twisted versions of the original image, which leads to a better model, 
# since it learns on the good and bad mix of images

train_datagen = ImageDataGenerator(
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)
 
# Defining pre-processing transformations on raw images of testing data
# No transformations are done on the testing images

test_datagen = ImageDataGenerator()
 
# Generating the Training Data
training_set = train_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=20,
        class_mode='categorical')
 
 
# Generating the Testing Data
test_set = test_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=20,
        class_mode='categorical')
 
# Printing class labels for each face
test_set.class_indices

Found 1145 images belonging to 24 classes.
Found 1145 images belonging to 24 classes.


{'dave': 0,
 'dev': 1,
 'donna': 2,
 'dori': 3,
 'eric': 4,
 'gervez': 5,
 'hui': 6,
 'john': 7,
 'kim': 8,
 'kori': 9,
 'loki': 10,
 'matt': 11,
 'nate': 12,
 'paaji': 13,
 'pamela': 14,
 'rinku': 15,
 'rinku1': 16,
 'roger': 17,
 'saisrujan': 18,
 'shreya': 19,
 'srikar': 20,
 'srujan': 21,
 'tom': 22,
 'van': 23}

### Creating lookup table for all faces:

In [209]:
# class_indices have the numeric tag for each face
TrainClasses=training_set.class_indices
 
# Storing the face and the numeric tag for future reference
ResultMap={}
for faceValue,faceName in zip(TrainClasses.values(),TrainClasses.keys()):
    ResultMap[faceValue]=faceName
 
 # Saving the face map for future reference
import pickle
with open("ResultsMap.pkl", 'wb') as fileWriteStream:
    pickle.dump(ResultMap, fileWriteStream)
 
 # The model will give answer as a ID or tag
# We show the corresponding face name for it using the mapping
print("Mapping of Face and its ID")
print('\n',ResultMap)
 
# The number of neurons for the output layer is equal to the number of faces
OutputNeurons=len(ResultMap)
print('\n Number of Output Neurons: ', OutputNeurons)

Mapping of Face and its ID

 {0: 'dave', 1: 'dev', 2: 'donna', 3: 'dori', 4: 'eric', 5: 'gervez', 6: 'hui', 7: 'john', 8: 'kim', 9: 'kori', 10: 'loki', 11: 'matt', 12: 'nate', 13: 'paaji', 14: 'pamela', 15: 'rinku', 16: 'rinku1', 17: 'roger', 18: 'saisrujan', 19: 'shreya', 20: 'srikar', 21: 'srujan', 22: 'tom', 23: 'van'}

 Number of Output Neurons:  24


## Creating a CNN deep learning model 


In [210]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense
 
#''Initializing the Convolutional Neural Network'''
model = Sequential()
 
''' STEP--1 Convolution
# Adding the first layer of CNN
# we are using the format (64,64,3) because we are using TensorFlow backend
# It means 3 matrix of size (64X64) pixels representing Red, Green and Blue components of pixels
'''
model.add(Convolution2D(32, kernel_size=(5, 5), strides=(1, 1), input_shape=(64,64,3), activation='relu'))
 
'''# STEP--2 MAX Pooling'''
model.add(MaxPool2D(pool_size=(2,2)))
 
'''############## ADDITIONAL LAYER of CONVOLUTION for better accuracy #################'''
model.add(Convolution2D(64, kernel_size=(5, 5), strides=(1, 1), activation='relu'))
 
model.add(MaxPool2D(pool_size=(2,2)))
 
'''# STEP--3 FLattening'''
model.add(Flatten())
 
'''# STEP--4 Fully Connected Neural Network'''
model.add(Dense(64, activation='relu'))
model.add(Dense(OutputNeurons, activation='softmax'))
 
'''# Compiling the CNN'''
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])
 
import time
# Measuring the time taken by the model to train
StartTime=time.time()
 
# Starting the model training
model.fit_generator(
                    training_set,
                    steps_per_epoch=15,
                    epochs=30,
                    validation_data=test_set,
                    validation_steps=10)
 
EndTime=time.time()
print("###### Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes ######')

Epoch 1/30
15/15 [==============================] - 2s 127ms/step - loss: 28.7825 - accuracy: 0.1733 - val_loss: 2.0316 - val_accuracy: 0.4600
Epoch 2/30
15/15 [==============================] - 2s 114ms/step - loss: 1.6263 - accuracy: 0.5833 - val_loss: 1.0184 - val_accuracy: 0.7200
Epoch 3/30
15/15 [==============================] - 2s 133ms/step - loss: 0.7701 - accuracy: 0.7933 - val_loss: 0.6972 - val_accuracy: 0.8250
Epoch 4/30
15/15 [==============================] - 2s 124ms/step - loss: 0.6291 - accuracy: 0.8500 - val_loss: 0.5655 - val_accuracy: 0.8800
Epoch 5/30
15/15 [==============================] - 2s 115ms/step - loss: 0.4742 - accuracy: 0.8807 - val_loss: 0.2603 - val_accuracy: 0.9500
Epoch 6/30
15/15 [==============================] - 2s 123ms/step - loss: 0.4289 - accuracy: 0.8933 - val_loss: 0.2082 - val_accuracy: 0.9300
Epoch 7/30
15/15 [==============================] - 2s 114ms/step - loss: 0.2162 - accuracy: 0.9333 - val_loss: 0.2016 - val_accuracy: 0.9700
Epoch

## Making single predictions 

In [211]:
import numpy as np
import cv2

from keras.preprocessing import image


ImagePath  = '/Users/sruja/Pictures/Camera Roll/pred1.jpg'

img = cv2.imread(ImagePath)

#########################################################################################################
# Convert into grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt2.xml')

# Detect faces
faces = face_cascade.detectMultiScale(gray, 1.1, 4)

for (x, y, w, h) in faces:
    cv2.rectangle(img, (x, y), (x+w, y+h),
                (0, 0, 255), 2)

    faces = img[y:y + h, x:x + w]
#         cv2.imshow("face",faces)
    cv2.imwrite(ImagePath, faces)

### The code commented in this block is used for a fresh image or an uncropped image.
#########################################################################################################

test_image = image.load_img(ImagePath,target_size=(64, 64))
test_image = image.img_to_array(test_image)

test_image = np.expand_dims(test_image,axis=0)

result = model.predict(test_image,verbose=0)

#print(result)

#if result>=1:
print('----'*10)
print('Prediction is: ',ResultMap[np.argmax(result)])
print('----'*10)
#else:
#    print("unknown")

----------------------------------------
Prediction is:  saisrujan
----------------------------------------
